### Многопоточность. Часть 2

<br />

##### race condition

https://en.wikipedia.org/wiki/Race_condition

__Вопросы__:
* что такое race condition?  
  Race condition - несинхронизированный доступ потоков к ресурсу, для которого синхронизация требуется.
* как с ним бороться?
* каковы гарантии стандарта языка С++ при возникновении race contidition?

__Пример__:

```c++
#include <cassert>
#include <iostream>
#include <thread>
#include <vector>

int parallel_sum(const std::vector<int>& v)
{
    int len = v.size();

    int rv = 0;
    
    std::thread t1([&](){
        for (int i = 0; i < len / 2; ++i)
            rv += v[i];
    });
    std::thread t2([&](){
        for (int i = len / 2; i < len; ++i)
            rv += v[i];
    });
    
    t1.join();
    t2.join();

    return rv;
}

int main()
{
    const std::vector<int> v(3'000'000, 1);
    std::cout << parallel_sum(v) << std::endl;
    return 0;
}
```

Возможный вывод:
    
```sh
$ clang++ -O2 test.cpp -lpthread && ./a.out
1510753
```

Как получается race condition:

```c++
thread_1:             | thread_2:
    read  rv          |     read  rv
    calc  rv + v[i1]  |     calc  rv + v[i2]
    write rv          |     write rv
```

<br />

##### mutex

MUTual EXclusive access primitive

https://en.cppreference.com/w/cpp/thread/mutex

Методы:
    
* `void lock()` - дождаться, пока mutex будет освобождён, и захватить его
* `void unlock()` - освободить захваченный mutex
* `bool try_lock()` - попытка захватить mutex, если он свободен
* `native_handle_type native_handle()` - ОС-специфичный handle (как можно здесь догадаться, mutex - объект ядра ОС)

Вариант исправления задачи с суммой через mutex:

```c++
#include <mutex>  // include for mutex

int parallel_sum(const std::vector<int>& v)
{
    int len = v.size();

    int rv = 0;
    std::mutex mtx; // synchronization primitive for |rv|

    std::thread t1([&](){
        for (int i = 0; i < len / 2; ++i) {
            mtx.lock();   // acquire resource
            rv += v[i];
            mtx.unlock(); // release resource
        }
    });
    std::thread t2([&](){
        for (int i = len / 2; i < len; ++i) {
            mtx.lock();   // acquire resource
            rv += v[i];
            mtx.unlock(); // release resource
        }
    });

    t1.join();
    t2.join();

    return rv;
}
```

Вывод:

```c++
$ clang++ -O2 test.cpp -lpthread && ./a.out
3000000
```

__Замечание__: т.к. переменная `unsigned len` является константой, то в поток её можно передать и по ссылке:
* Константные объекты, имеющиеся до создания фонового потока, можно читать без опасения на race condition
* `unsigned` - маленький объект, его проще передать по копии, чем по ссылке

__Для обсуждения__:
    
1. Никогда не делайте частый доступ до int-переменной через mutex:
    * проблема kernel space
    * более дешёвые альтернативы
    * алгоритм может быть реализован с меньшим числом синхронизаций
2. Какая проблема с парными вызовами `lock()/unlock()`? Подсказка: то же самое что и с `new/delete`?

<br />

##### lock_guard

https://en.cppreference.com/w/cpp/thread/lock_guard

`std::lock_guard` - RAII обёртка над `std::mutex::lock/unlock`.

В конструкторе захватывает mutex, в деструкторе его освобождает.

Второй вариант параллельного суммирования:
    
```c++
#include <mutex>

int parallel_sum(const std::vector<int>& v)
{
    int len = v.size();

    int rv = 0;
    std::mutex mtx;

    std::thread t1([&](){
        for (int i = 0; i < len / 2; ++i) {
            std::lock_guard<std::mutex> guard(mtx);  // RAII-style for resource locking
            rv += v[i];
        }
    });
    std::thread t2([&](){
        for (int i = len / 2; i < len; ++i) {
            std::lock_guard<std::mutex> guard(mtx);  // RAII-style for resource locking
            rv += v[i];
        }
    });

    t1.join();
    t2.join();

    return rv;
}
```

<br />

##### Thread-safe - объекты (объекты с доступом из нескольких потоков)

Вариант организации многопоточной очереди:
    
```c++
template<typename T>
class MTQueue
{
public:
    std::optional<T> pop() {
        std::lock_guard<std::mutex> guard(mtx);
        
        if (queue.empty())
            return std::nullopt;
        
        T x = queue.back();
        queue.pop_back();
        return x;
    }
    
    void push(T x) {
        std::lock_guard<std::mutex> guard(mtx);        
        queue.push_front(std::move(x));        
    }
    
    MTQueue() = default;    
    
    MTQueue(const MTQueue& rhs) {
        std::lock_guard<std::mutex> guard(rhs.mtx);
        queue = rhs.queue;
    }

    MTQueue(MTQueue&& rhs) noexcept {
        std::lock_guard<std::mutex> guard(rhs.mtx);
        queue = std::move(rhs.queue);
    }
    
    ~MTQueue() noexcept {
        std::lock_guard<std::mutex> guard(mtx);
        queue.clear();
    }

    // мы ещё не готовы реализовать присваивание двух объектов,
    // живущих на разных потоках
    MTQueue& operator = (const MTQueue& rhs) = delete;
    MTQueue& operator = (MTQueue&& rhs) noexcept = delete;
    
private:
    std::deque<T> queue;
    std::mutex mtx;
};
```

__Вопрос__: где в этом коде происходит ужас и кошмар?

<details>
<summary>ответ</summary>
<p>

```c++
~MTQueue() noexcept {
    std::lock_guard<std::mutex> guard(rhs); // <---- тут
    queue.clear();
}
```

Обсудить: если этот `guard` оказался нужен, то что происходит в программе в этот момент?

Пример: перемежающиеся вызовы `push` и `~MTQueue`.
   
</p>
</details>

```c++
MTQueue<int> m


thread_1:           | thread_2:
    m.~MTQueue()    |     m.push(5)
        lock_guard  |         lock_guard
        processing  |         processing
```

__Вопрос__: почему бессмысленно иметь метод `empty`?

Рассмотрим такой вариант:
    
```c++
class MTQueue
{
    ...;
    
    bool empty() {
        std::lock_guard<std::mutex> guard(mtx);

        return queue.empty();
    }
    
    T pop() {
        std::lock_guard<std::mutex> guard(mtx);

        T x = queue.back();
        queue.pop_back();
        return x;
    }
}
```

И вот такой код, исполняющийся параллельно двумя потоками:

```c++
void process_queue(MTQueue& q)   |    void process_queue(MTQueue& q)
{                                |    {
    if (!q.empty())              |        if (!q.empty())
        process_item(q.pop());   |            process_item(q.pop());
}                                |    }
```

Дадим потокам на вход одну и ту же очередь с одним элементов.

<br />

__Упражнение 1__: найдите ошибку в коде. Почему это является ошибкой? Приведите пример.

```c++
class MTSearcher {
private:
    std::mutex mtx;
    int max_search_result_size;
   
public:
    int get_max_search_result_size() const {
        return max_search_result_size;
    }
    
    void set_max_search_result_size(int size) {
        std::lock_guard<std::mutex> guard(mtx);
        max_search_result_size = size;
    }
};
```


<br />

__Упражнение 2__: найдите ошибку в коде


```c++
template<typename T>
class MTQueue
{
private:
    std::mutex mtx;
    std::queue<std::shared_ptr<T>> queue;

public:
    std::shared_ptr<T>& peek() noexcept
    {
        std::lock_guard<std::mutex> guard(mtx);
        return queue.back();
    }
    
    void push(std::shared_ptr<T> x)
    {
        std::lock_guard<std::mutex> guard(mtx);
        queue.push_back(x);
    }
};

// thread 1:
mt_queue.peek() = nullptr;

// thread 2:
std::cout << *mt_queue.peak();
```


Редко когда многопоточный класс может позволить себе такую роскошь как возвращение ссылок/указателей на данные. Как правило, нужно делать defensive copies:

```c++
class MTQueue {
    ...
        
    std::shared_ptr<T> peek() noexcept
    {
        std::lock_guard<std::mutex> guard(mtx);
        return queue.back();
    }
};
```

<br />

__Рекомендация__: минимизируйте число классов и объектов в программе, поддерживающих многопоточный доступ. Классы с поддержкой многопоточности сложнее проектировать и сложнее в использовании.

<br />

##### deadlock

https://en.wikipedia.org/wiki/Deadlock

__Вопрос__: что такое deadlock и как с ним бороться?

deadlock классический:
    
```c++
std::mutex m1; // мьютекс для ресурса 1
std::mutex m2; // мьютекс для ресурса 2

void worker_1()                  |  void worker_2()
{                                |  {
    std::lock_guard guard1(m1);  |      std::lock_guard guard2(m2);
    std::lock_guard guard2(m2);  |      std::lock_guard guard1(m1);
    ...;                         |      ...;
}                                |  }
```

__Вопрос__: как его починить?

__Ответ__:

```c++
std::mutex m1; // мьютекс для ресурса 1
std::mutex m2; // мьютекс для ресурса 2

void worker_1()                  |  void worker_2()
{                                |  {
    std::lock_guard guard1(m1);  |      std::lock_guard guard1(m1);
    std::lock_guard guard2(m2);  |      std::lock_guard guard2(m2);
    ...;                         |      ...;
}                                |  }
```

<br />

##### scoped_lock

Не всегда легко определить правильный порядок блокировки мьютексов.

Вспомним `MTQueue` и его `operator =`:

```c++
template<typename T>
class MTQueue
{
private:
    std::mutex mtx;
    std::queue<T> queue;

public:
    MTQueue& operator =(const MTQueue& rhs)
    {
        if (this != &rhs)
        {
            // что блокировать первым?
            // std::lock_guard guard_1(mtx);
            // std::lock_guard guard_2(rhs.mtx);
            queue = rhs.queue;
        }
        return *this;
    }
};

MTQueue<int> q1;
MTQueue<int> q2;

// thread 1:
q2 = q1;

// thread 2:
q1 = q2;
```

или, например:

```c++
bool operator == (const MTQueue& lhs, const MTQueue& rhs)
{
    // что блокировать первым?
    // std::lock_guard guard_1(lhs.mtx);
    // std::lock_guard guard_2(rhs.mtx);
    return lhs.queue == rhs.queue;
}

// thread 1:
q1 == q2


// thread 2:
q2 == q1
```

В таком коде нет правильного выбора последовательности блокировок

<br />

Для решения проблемы существует `std::scoped_lock`:

https://en.cppreference.com/w/cpp/thread/scoped_lock

```c++
bool operator == (const MTQueue& lhs, const MTQueue& rhs)
{
    std::scoped_lock guard(lhs.mtx, rhs.mtx);

    return lhs.queue == rhs.queue;
}
```

и:

```c++
template<typename T>
class MTQueue
{
private:
    std::mutex mtx;
    std::queue<T> queue;

public:
    MTQueue& operator =(const MTQueue& rhs)
    {
        if (this != &rhs)
        {
            std::scoped_lock guard(mtx, rhs.mtx);

            queue = rhs.queue;
        }
        return *this;
    }
};
```

<br />

Для варианта:

```c++
std::mutex m1;
std::mutex m2;

std::scoped_lock guard_1{m1, m2};  // thread 1
std::scoped_lock guard_2{m2, m1};  // thread 2
```

`std::scoped_lock` автоматически определяет порядок, таким образом, что `guard_1{m1, m2}` и `guard_2{m2, m1}` на разных потоках "эквивалентны" и не приведут к deadlock.

**Вопрос:** Как это можно было бы реализовать?

<br />

**Практические упражнения**

* `num_threads` потоков параллельно ищут все вхождения элемента в массиве,  
  когда находят, выводят в `std::cout` строку `"thread {thread_num} has found {element} at index {ix}"`
* Решить предыдущую задачу без использования `std::mutex`
* Найти наиболее часто встречающийся символ в строке в несколько потоков. Для решения использовать 256 счётчиков и 256 мьютексов.
* Напишите многопоточную очередь `MTQueue` с ограничением в 10 элементов в очереди.
* Напишите программу, в которой:
  * один фоновый поток кладёт в `MTQueue` 20 элементов и завершается
  * второй фоновый достаёт их из `MTQueue` и распечатывает.
  * главный поток через 3 секунды работы приложения сигнализирует второму, что нужно завершиться ("сигнализирование" можно сделать через bool-переменную, которую фоновый поток будет читать).

<br />

##### Резюме

* `std::mutex` - один из вариантов избавления от race condition
* захватывать и освобождать `std::mutex` желательно через RAII: `std::lock_guard`
* объекты с многопоточным доступом требуют более тщательной проработки дизайна
* `std::scoped_lock` как вариант лечения deadlock